In [4]:
import requests

url = "http://localhost:8080/resources/read?uri=resource://sample-csv"

response = requests.get(url)

print(f"Status code: {response.status_code}")
if response.status_code == 200:
    print("Resource content preview (first 500 chars):")
    print(response.text[:500])
else:
    print("Failed to fetch resource.")


Status code: 404
Failed to fetch resource.


In [16]:
import requests
import json

def parse_sse_response(response_text):
    """Parse Server-Sent Events response to extract JSON data."""
    lines = response_text.strip().split('\n')
    for line in lines:
        if line.startswith('data: '):
            json_data = line[6:]  # Remove 'data: ' prefix
            return json.loads(json_data)
    return None

# Replace your original code with this:
payload = {
    "jsonrpc": "2.0",
    "id": 1,
    "method": "resources/read",
    "params": {"uri": "resource://sample-csv"}
}

response = requests.post("http://localhost:8080/mcp",
                       json=payload,
                       headers={
                           "Content-Type": "application/json",
                           "Accept": "application/json, text/event-stream"
                       })

if response.status_code == 200:
    result = parse_sse_response(response.text)
    if result and 'result' in result and 'contents' in result['result']:
        csv_content = result['result']['contents'][0]['text']
        print("Resource content:", csv_content)
    else:
        print("Unexpected response format")
else:
    print(f"Failed to fetch resource. Status code: {response.status_code}")

Resource content: region,revenue
North,1000
South,1500
East,900
West,1300
North,1100
South,1600
East,950
West,1250



In [10]:
encoded_uri

'resource%3A%2F%2Fsample-csv'

In [15]:
import requests
import json

def parse_sse_response(response_text):
    """Parse Server-Sent Events response to extract JSON data."""
    lines = response_text.strip().split('\n')
    for line in lines:
        if line.startswith('data: '):
            json_data = line[6:]  # Remove 'data: ' prefix
            return json.loads(json_data)
    return None

def get_fastmcp_resource(uri):
    """Get a resource from FastMCP server."""
    payload = {
        "jsonrpc": "2.0",
        "id": 1,
        "method": "resources/read",
        "params": {"uri": uri}
    }
    
    response = requests.post("http://localhost:8080/mcp",
                           json=payload,
                           headers={
                               "Content-Type": "application/json",
                               "Accept": "application/json, text/event-stream"
                           })
    
    if response.status_code == 200:
        # Parse the SSE response
        result = parse_sse_response(response.text)
        if result and 'result' in result and 'contents' in result['result']:
            return result['result']['contents'][0]['text']
        else:
            print("Unexpected response format:", result)
            return None
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

def list_fastmcp_resources():
    """List all available resources from FastMCP server."""
    payload = {
        "jsonrpc": "2.0",
        "id": 1,
        "method": "resources/list",
        "params": {}
    }
    
    response = requests.post("http://localhost:8080/mcp",
                           json=payload,
                           headers={
                               "Content-Type": "application/json",
                               "Accept": "application/json, text/event-stream"
                           })
    
    if response.status_code == 200:
        result = parse_sse_response(response.text)
        if result and 'result' in result and 'resources' in result['result']:
            return result['result']['resources']
        else:
            print("Unexpected response format:", result)
            return None
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

if __name__ == "__main__":
    print("=== Available Resources ===")
    resources = list_fastmcp_resources()
    if resources:
        for resource in resources:
            print(f"Name: {resource['name']}")
            print(f"URI: {resource['uri']}")
            print(f"Description: {resource['description']}")
            print("---")
    
    print("\n=== Getting CSV Content ===")
    csv_content = get_fastmcp_resource("resource://sample-csv")
    if csv_content:
        print("CSV Content:")
        print(csv_content)

=== Available Resources ===
Name: get_sample_csv
URI: resource://sample-csv
Description: Returns the content of sample.csv.
---

=== Getting CSV Content ===
CSV Content:
region,revenue
North,1000
South,1500
East,900
West,1300
North,1100
South,1600
East,950
West,1250



In [25]:
import requests
import json

# The MCP JSON-RPC endpoint your server exposes (matches server.py run config)
MCP_RPC_URL = "http://127.0.0.1:8080/mcp/rpc"

# Request headers:
# - Content-Type: tell server we're sending JSON
# - Accept: we accept either plain JSON or SSE (text/event-stream); server may choose
# - MCP-Protocol-Version: version hint some servers expect
HEADERS = {
    "Content-Type": "application/json",
    "Accept": "application/json, text/event-stream",
    "MCP-Protocol-Version": "2025-06-18",
}


payload = {
        "jsonrpc": "2.0",
        "method": "tools/list",
        "params": None or {},
        "id": 1,
    }

# Fire the HTTP POST to the JSON-RPC endpoint
resp = requests.post(MCP_RPC_URL, json=payload, headers=HEADERS)



In [26]:
body = resp.text.strip()
body

'event: message\r\ndata: {"jsonrpc":"2.0","id":5000,"result":{"tools":[{"name":"echo","inputSchema":{"properties":{"text":{"title":"Text","type":"string"}},"required":["text"],"type":"object"},"outputSchema":{"properties":{"result":{"title":"Result","type":"string"}},"required":["result"],"title":"_WrappedResult","type":"object","x-fastmcp-wrap-result":true},"_meta":{"_fastmcp":{"tags":[]}}},{"name":"summarize_csv","description":"Load CSV with pandas and send summary request to OpenAI.","inputSchema":{"properties":{"csv_text":{"title":"Csv Text","type":"string"}},"required":["csv_text"],"type":"object"},"outputSchema":{"properties":{"result":{"title":"Result","type":"string"}},"required":["result"],"title":"_WrappedResult","type":"object","x-fastmcp-wrap-result":true},"_meta":{"_fastmcp":{"tags":[]}}}]}}'